In [32]:
from core import database
import isb
import pandas as pd
import numpy as np
import h3
import datetime
import typing as t
import plotly.express as px
import plotly.graph_objects as go

mapbox_access_token = open(".mapbox_token").read()

In [33]:
db = database()

C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\auth\_default.py:76: UserWarning:

Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 



In [34]:
df_sale_agreement_times = db.query("""
select 
    sa1.sale_agreement_id, 
    sum(
        least(sa1.registration_date, upper(l1.span))::date -
        least(sa1.registration_date, lower(l1.span))::date
    ) as listing_days
from listings l1
join sale_agreements sa1
on l1.realestate_id = sa1.realestate_id
and l1.span && daterange((registration_date - make_interval(months:=12))::date, registration_date::date, '[]')
group by sa1.sale_agreement_id
""")

In [35]:
df_sales = db.query("""
select 
  s.registration_date::date,
  date_trunc('quarter', s.registration_date::date) as registration_month,
  s.sale_agreement_id,
  r.realestate_id, 
  u.district_id,
  r.square_meters, 
  r.rooms,
  r.construction_year,
  rt.name as realestate_type_name, 
    case when rt.name in ('Fjölbýlishús', 'Hæð') then 'Fjölbýli' else 'Sérbýli' end as realestate_category_name,
  c.name as city_name,
  a.name as address_name,
  a.street_name,
  case when construction_year >= extract(year from now()) - 5 and (sa.first_sale_date is null or sa.first_sale_date < s.registration_date) then 'new' else 'old' end as construction_category,
  purchase_price, 
  (select price from listing_prices lp where lp.span && daterange((s.registration_date - make_interval(months:=12))::date, s.registration_date::date, '[]') and lp.realestate_id = s.realestate_id order by span desc limit 1) as listing_price,
  --ld.listing_days,
  ST_X(u.origin) as latitude, 
  ST_Y(u.origin) as longitude
from sale_agreements s
join realestates r
on s.realestate_id = r.realestate_id
left join (
  select realestate_id, min(registration_date) as first_sale_date
  from sale_agreements
  group by realestate_id         
) sa
on r.realestate_id = sa.realestate_id
join realestate_types rt
on r.realestate_type_id = rt.realestate_type_id
join units u 
on r.unit_id = u.unit_id
join addresses a
on u.address_id = a.address_id
join lands la
on a.land_id = la.land_id
join postals p
on la.postal_id = p.postal_id
join cities c
on p.city_id = c.city_id
join regions reg
on c.region_id = reg.region_id
where rt.name in ('Fjölbýlishús', 'Par/Raðhús', 'Hæð', 'Einbýlishús')
  and reg.name = 'Norðurland eystra'
  and c.name = 'Akureyrarbær'
  and is_valid_agreement             
""")\
.assign(sqm_price = lambda r: r.purchase_price / r.square_meters)\
.merge(df_sale_agreement_times, how='left')

In [36]:
df_sales[~df_sales.listing_price.isnull()]

,registration_date,registration_month,sale_agreement_id,realestate_id,district_id,square_meters,rooms,construction_year,realestate_type_name,realestate_category_name,city_name,address_name,street_name,construction_category,purchase_price,listing_price,latitude,longitude,sqm_price,listing_days
1,2025-01-14,2025-01-01 00:00:00+00:00,937fb8a9-2da3-4a3f-bb44-cff3c1846801,0036dfe8-548b-4435-a45a-6c56857faf51,None,85.8,3.0,2024.0,Fjölbýlishús,Fjölbýli,Akureyrarbær,Austurbrú 12,Austurbrú,old,75500000,88000000.0,-18.088203,65.679290,879953.379953,11.0
2,2020-12-22,2020-10-01 00:00:00+00:00,77d3b03b-31d9-48fa-952d-019737dbcc78,003caaed-2812-4968-83ac-56005984ac5f,None,53.6,2.0,2017.0,Fjölbýlishús,Fjölbýli,Akureyrarbær,Austurbrú 2,Austurbrú,old,30000000,39500000.0,-18.087969,65.680405,559701.492537,845.0
6,2023-07-17,2023-07-01 00:00:00+00:00,53af710c-e820-4f13-ade3-ad1f938a94c8,0069674e-1d19-410a-b745-9af6a24c9c81,None,202.1,4.0,1967.0,Par/Raðhús,Sérbýli,Akureyrarbær,Langholt 24,Langholt,old,85800000,89800000.0,-18.113146,65.694280,424542.305789,129.0
7,2024-09-13,2024-07-01 00:00:00+00:00,ac840381-f343-42b1-b10a-5df1e324067e,007e4e42-5f66-45f0-95a2-8d1d9a1cc741,None,116.4,3.0,1999.0,Par/Raðhús,Sérbýli,Akureyrarbær,Urðargil 13,Urðargil,old,81300000,78500000.0,-18.143050,65.689095,698453.608247,48.0
8,2018-07-20,2018-07-01 00:00:00+00:00,63e95980-ab25-4dfe-ab7f-486a32d9a88a,007e4e42-5f66-45f0-95a2-8d1d9a1cc741,None,116.4,3.0,1999.0,Par/Raðhús,Sérbýli,Akureyrarbær,Urðargil 13,Urðargil,old,49000000,50900000.0,-18.143050,65.689095,420962.199313,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9861,2019-09-17,2019-07-01 00:00:00+00:00,ad0986aa-4af8-46bf-bc65-81b2e98ba624,fe5ad388-c2e7-4534-951d-5d4cd7dcc357,None,62.2,2.0,2017.0,Fjölbýlishús,Fjölbýli,Akureyrarbær,Undirhlíð 1,Undirhlíð,old,29800000,31900000.0,-18.107815,65.692696,479099.678457,131.0
9863,2025-10-09,2025-10-01 00:00:00+00:00,f170099a-4943-4109-8e96-680eb79d8d5f,fe76bfca-115c-4be4-81b9-8a8912113a50,None,90.5,3.0,1948.0,Fjölbýlishús,Fjölbýli,Akureyrarbær,Þórunnarstræti 106,Þórunnarstræti,old,54500000,54500000.0,-18.099445,65.680080,602209.944751,48.0
9864,2018-02-16,2018-01-01 00:00:00+00:00,91969e12-2991-471a-9f64-5d637f5ee516,fec2a77f-bf28-4372-b709-08ba9c0ce944,None,234.7,5.0,2009.0,Einbýlishús,Sérbýli,Akureyrarbær,Krókeyrarnöf 6,Krókeyrarnöf,old,75400000,77400000.0,-18.088854,65.662910,321261.184491,118.0
9871,2021-06-11,2021-04-01 00:00:00+00:00,905ea4f8-02f9-4d6f-b475-a0c221c79ec1,ffae5631-2aa4-4e22-b20b-d3ea6c63d61a,None,86.6,3.0,2017.0,Fjölbýlishús,Fjölbýli,Akureyrarbær,Ásatún 46,Ásatún,old,40800000,41900000.0,-18.097775,65.666960,471131.639723,38.0


In [37]:
def make_plot(df):

    fig_colors = isb.Figure()
    fig = isb.Figure('core')
    
    position = 0

    for i, column in enumerate(df.columns):

        current_pos = df[column].iloc[-1]

        fig.add_scatter(
            x = df.index,
            y = df[column],
            stackgroup='one',
            line = dict(
                color = fig.font_color if i < 2 else fig.bg_color,
                dash = 'solid',
                width = 1
            ),
            fillcolor = fig_colors.colors(i, 0.6),
            fill='tonexty',
            showlegend=False
        )

        text = dict(
            above_listing=f"<b>{current_pos:.1%}</b><br><i>yfir</i>",
            on_listing=f"<b>{current_pos:.1%}</b><br><i>á ásettu</i>",
            below_listing=f"<b>{current_pos:.1%}</b><br><i>undir</i>",
        )[column]
            
        fig.add_annotation(
            xref='paper',
            yref='y',
            x = 1.04,
            y = (position) + current_pos/2,
            showarrow=False,
            text=text,
            font = fig.get_font(size=16, color = fig.font_color),
            xanchor='center'
        )

        position += current_pos
        print(position)

    fig.update_yaxes(
        tickformat=".0%",
        dtick=.1
    )

    fig.set_title(
        fig_colors.color_string("Hlutfall kaupa sem seljast <color 2>undir</color>, <color 1>á</color> eða <color 0>yfir</color> ásettu verði"),
    )

    fig.add_annotation(
        x = -0.027,
        y = 1.13,
        showarrow=False,
        font = fig.get_font(size=18),
        text = "<i>Aðeins þeir kaupsamningar þar sem til er fasteignaauglýsing á tímabilinu 12 mánuðum fyrir undirritaðan sölusamning.</i>",
        xref='paper', yref='paper',
        xanchor='left',
        align='left'

    )

    fig.update_layout(
        margin_t = 150,
        #title_y = 0.93
    )

    fig.add_logo()


    return fig


fig = df_sales\
.loc[lambda r: ~r.listing_price.isnull()]\
.loc[lambda r: r.listing_price > 0]\
.loc[lambda r: r.listing_price > 10e6]\
.loc[lambda r: r.purchase_price < 300e6]\
.assign(
    above_listing = lambda r: r.purchase_price > r.listing_price,
    on_listing = lambda r: r.purchase_price == r.listing_price,
    below_listing = lambda r: r.purchase_price < r.listing_price,
)\
.groupby(['registration_month'])\
.agg(
    above_listing = ('above_listing', 'mean'),
    on_listing = ('on_listing', 'mean'),
    below_listing = ('below_listing', 'mean'),
)\
.pipe(make_plot)


fig\
.export('below-on-above', scale=1.5)

fig.show()

0.2328767123287671
0.4520547945205479
1.0


In [6]:
def get_sales_filtered_by(realestate_category_name: str='Fjölbýli', construction_category: str='new', city_name: str=''):
    return df_sales\
        .loc[lambda r: r.realestate_category_name == realestate_category_name]\
        .loc[lambda r: r.construction_category == construction_category]\
        .loc[lambda r: r.square_meters >= 20]\
        .loc[lambda r: r.square_meters < 250]

def get_listings_filtered_by(realestate_category_name: str='Fjölbýli', construction_category: str='new', city_name: str=''):
    return df_listings\
        .loc[lambda r: r.realestate_category_name == realestate_category_name]\
        .loc[lambda r: r.construction_category == construction_category]\
        .loc[lambda r: r.square_meters >= 20]\
        .loc[lambda r: r.square_meters < 250]

In [7]:
fig = isb.Figure()

df_hfj_sale = get_sales_filtered_by(city_name='Reykjavíkurborg')

fig.add_scatter(
    y = df_hfj_sale.city_name.apply(lambda x: 1.1 + np.random.random()/100),
    x = df_hfj_sale.square_meters,
    mode = 'markers',
    marker = dict(
        color = fig.colors(1)
    )
)
df_hfj_list = get_listings_filtered_by(city_name='Reykjavíkurborg')

fig.add_scatter(
    y = df_hfj_list.city_name.apply(lambda x: 1.1 + np.random.random()/100),
    x = df_hfj_list.square_meters,
    mode = 'markers',
    marker = dict(
        color = fig.colors(0)
    )
)

NameError: name 'df_listings' is not defined

In [10]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt



def fit_dist(values: pd.Series, x: np.array, print_results: bool=False) -> pd.DataFrame:

    distributions = {
        "normal": stats.norm,
        "lognorm": stats.lognorm,
        "gamma": stats.gamma,
        "weibull": stats.weibull_min
    }

    results = {}

    for name, dist in distributions.items():
        # Fit distribution to data
        params = dist.fit(values)
        
        # Compute log-likelihood
        loglik = np.sum(dist.logpdf(sqm, *params))
        
        # Calculate AIC = 2k - 2logL
        k = len(params)
        aic = 2 * k - 2 * loglik

        results[name] = {"params": params, "loglik": loglik, "aic": aic}

    best_dist = min(results, key=lambda x: results[x]["aic"])

    if print_results:
        print("Best fitting distribution:", best_dist)
        print("Parameters:", results[best_dist]["params"])

    dist = distributions[best_dist]
    params = results[best_dist]["params"]

    pdf = dist.pdf(x, *params)

    return pd.DataFrame(dict(
        x = x,
        pdf = pdf
    ))

In [12]:

def make_plot(df, title="Hlutfall eigna <color 0>á söluskrá</color> og <color 2>seldra</color> (ekki nýbyggingar)"):

    fig = isb.Figure('core')

    xs = df.index.to_series().to_list()[1:-1]

    for i, column in enumerate(df.columns):

        color_i = i*2 + 1

        fig.add_scatter(
            x = df.index,
            y = df[column],
            line = dict(
                color = fig.colors(color_i),
                width = 3
            ),
            fillcolor = fig.colors(color_i, .20),
            fill = 'tozeroy',
            showlegend=False
        )

        xs = df.index.to_series()
        ys = df[column]

        average = np.average(xs, weights=ys)
        position = df.loc[round(average)]

        fig.add_scatter(
            x = [average]*2,
            y = [0, position[column]],
            mode = 'lines',
            line = dict(
                color = fig.colors(color_i),
                dash = 'dash',
                width = 3
            ),
            showlegend=False
        )

        names = dict(
            pdf_list = f"Eignir á sölu<br><b>{round(average, 1)} m²</b><br>að meðaltali",
            pdf_sale = f"Seldar eignir<br><b>{round(average, 1)} m²</b><br>að meðaltali"
        )

        fig.add_annotation(
            x = average,
            y = position[column],
            text = names[column],
            showarrow=True,
            arrowsize=2,
            ax = 30 * (1 if i == 0 else -1),
            ay = -80,
            yshift=2.5,
            bgcolor = fig.bg_color,
            arrowcolor=fig.font_color,
            font = fig.get_font(size=16)
        )

        
        fig.add_scatter(
            x = [average],
            y = [position[column]],
            mode = 'markers',
            marker = dict(
                color = fig.colors(color_i),
                size=8
            ),
            showlegend=False
        )
        fig.add_scatter(
            x = [average],
            y = [position[column]],
            mode = 'markers',
            marker = dict(
                color = fig.bg_color,
                size=4
            ),
            showlegend=False
        )


        print(np.average(xs, weights=ys))

    fig.update_yaxes(
        tickformat = '.1%'
    )

    
    fig.set_title(
        title,
    )

    fig.add_annotation(
        x = -0.027,
        y = 1.13,
        showarrow=False,
        font = fig.get_font(size=18),
        text = "<i>Sölur miðast við allar sölur á árinu 2025, eignir á söluskrá miðast við 23. nóvember 2025</i>",
        xref='paper', yref='paper',
        xanchor='left',
        align='left'

    )

    fig.update_layout(
        margin_t = 150,
        #title_y = 0.93
    )

    fig.add_logo()

    return fig


df_hfj_sale = get_sales_filtered_by(construction_category='new')
df_hfj_list = get_listings_filtered_by(construction_category='new')

fit_dist(df_hfj_list.square_meters, x=np.arange(0, 200, 1)).set_index('x')\
.merge(
    fit_dist(df_hfj_sale.square_meters, x=np.arange(0, 200, 1)).set_index('x'),
    right_index=True, left_index=True,
    suffixes=["_list", "_sale"]
)\
.pipe(make_plot, title="Hlutfall eigna <color 1>á söluskrá</color> og <color 3>seldra</color> (aðeins nýbyggingar)")\
.export("04-sales-vs-listing--square-meters--new-construction", scale=1.5)


df_hfj_sale = get_sales_filtered_by(construction_category='old')
df_hfj_list = get_listings_filtered_by(construction_category='old')

fit_dist(df_hfj_list.square_meters, x=np.arange(0, 200, 1)).set_index('x')\
.merge(
    fit_dist(df_hfj_sale.square_meters, x=np.arange(0, 200, 1)).set_index('x'),
    right_index=True, left_index=True,
    suffixes=["_list", "_sale"]
)\
.pipe(make_plot, title="Hlutfall eigna <color 1>á söluskrá</color> og <color 3>seldra</color> (ekki nýbyggingar)")\
.export("04-sales-vs-listing--square-meters--old-construction", scale=1.5)

NameError: name 'df_listings' is not defined

In [ ]:

df_hfj_sale = get_sales_filtered_by(construction_category='new')
df_hfj_list = get_listings_filtered_by(construction_category='new')


fit_dist(df_hfj_list[(df_hfj_list.sqm_price > 300e3) & (df_hfj_list.sqm_price < 2000e3)].sqm_price, x=np.arange(0, 1.5e6, 10e3)).set_index('x')\
.merge(
    fit_dist(df_hfj_sale[(df_hfj_sale.sqm_price > 300e3) & (df_hfj_sale.sqm_price < 2000e3)].sqm_price, x=np.arange(0, 1.5e6, 10e3)).set_index('x'),
    right_index=True, left_index=True,
    suffixes=["_list", "_sale"]
)\
.isb.plot()

In [ ]:
import statsmodels.api as sm

In [ ]:
df_temp.head()

,realestate_id,listing_id,listing_span,listing_price_span,district_id,square_meters,rooms,construction_year,realestate_type_name,realestate_category_name,city_name,address_name,street_name,construction_category,price,days_listed,latitude,longitude,sqm_price
7,05c6c8d6-d0de-4840-8013-74d8b4a764b8,b6130605-180a-45e0-afc9-a2ef3d0b0e0e,"[2025-05-22,2025-11-24)","[2025-06-03,2025-11-24)",b7b8ed8c-8a80-4423-9de9-041822d94fd4,98.6,4.0,2024.0,Fjölbýlishús,Fjölbýli,Reykjavíkurborg,Vesturgata 66,Vesturgata,new,101500000,185,-21.954035,64.151924,1.029412e+06
8,062c0efe-0bfc-4010-bf68-c70d98ecea62,6eaf22f8-2bd3-45a2-bddf-c810e4331615,"[2024-03-07,2025-11-24)","[2024-09-17,2025-11-24)",b7b8ed8c-8a80-4423-9de9-041822d94fd4,103.1,3.0,2023.0,Fjölbýlishús,Fjölbýli,Reykjavíkurborg,Ánanaust 3,Ánanaust,new,99500000,626,-21.953815,64.152330,9.650824e+05
13,0ac36309-a49f-4504-876e-38018c5d2c71,f0a4c9b9-b394-403f-abe1-395d82cc3a50,"[2024-08-28,2025-11-24)","[2025-05-17,2025-11-24)",b7b8ed8c-8a80-4423-9de9-041822d94fd4,108.3,3.0,2024.0,Fjölbýlishús,Fjölbýli,Reykjavíkurborg,Sólvallagata 79,Sólvallagata,new,119900000,452,-21.958588,64.150604,1.107110e+06
16,0cf98bca-98e0-49c4-89aa-7cc07c06d857,634c833c-6a2d-4cd4-8cfb-5bd31997df63,"[2025-05-22,2025-11-24)","[2025-06-03,2025-11-24)",b7b8ed8c-8a80-4423-9de9-041822d94fd4,135.7,4.0,2024.0,Fjölbýlishús,Fjölbýli,Reykjavíkurborg,Vesturgata 66,Vesturgata,new,142900000,185,-21.954035,64.151924,1.053058e+06
21,0f91b371-1d3a-4511-a8e5-7488fc132972,8ac48393-4163-490c-8df9-db1d3a36d72a,"[2024-03-11,2025-11-24)","[2024-03-11,2025-11-24)",b7b8ed8c-8a80-4423-9de9-041822d94fd4,181.8,3.0,2024.0,Fjölbýlishús,Fjölbýli,Reykjavíkurborg,Mýrargata 43,Mýrargata,new,259900000,622,-21.952679,64.152490,1.429593e+06


In [ ]:
df_sale_centers = df_sales\
.groupby(['city_name'])\
.agg(
    latitude = ('latitude', 'mean'),
    longitude = ('longitude', 'mean')
)

In [ ]:
df_listing_centers = df_listings\
.groupby(['city_name'])\
.agg(
    latitude = ('latitude', 'mean'),
    longitude = ('longitude', 'mean')
)

In [ ]:
colors = isb.plot.helpers.create_color_scaler(
    min_value=0, 
    max_value=360,
    color_low="#279638",
    color_high="#fd5a54"
)
fig = isb.Figure()

fig.add_scattermapbox(
    lat=df_listing_centers.longitude, 
    lon=df_listing_centers.latitude, 
    #z=df.days_listed.apply(lambda x: 360 if x > 360 else x),
    #radius=5,
    #opacity=0.5
)
fig.add_scattermapbox(
    lat=df_sale_centers.longitude, 
    lon=df_sale_centers.latitude, 
    #z=df.days_listed.apply(lambda x: 360 if x > 360 else x),
    #radius=5,
    #opacity=0.5
)

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        style="light",
        center=dict(
            lat=64.1024998,
            lon=-21.897828
        ),
        pitch=0,
        zoom=10
    ),
)

fig.show()

In [ ]:
df_listings\
.loc[lambda r: r.square_meters < 300]\
.loc[lambda r: r.realestate_category_name == 'Fjölbýli']\
.loc[lambda r: r.construction_category == 'new']\
.assign(sqm_price = lambda r: r.price / r.square_meters)\
.groupby(['realestate_category_name', 'city_name', 'construction_category'])\
.agg(
    square_meters = ('square_meters', 'mean'),
    rooms = ('rooms', 'mean'),
    construction_year = ('construction_year', 'mean'),
    price = ('price', 'mean'),
    total = ('realestate_id', 'count'),
    sqm_price = ('sqm_price', 'mean')
)

square_meters  \
realestate_category_name city_name               construction_category                  
Fjölbýli                 Garðabær                new                       124.804630   
                                                 old                       120.919672   
                         Hafnarfjarðarkaupstaður new                        99.571899   
                                                 old                        98.196552   
                         Kópavogsbær             new                       105.545536   
                                                 old                       114.467568   
                         Mosfellsbær             new                        93.566667   
                                                 old                       102.468421   
                         Reykjavíkurborg         new                       105.063288   
                                                 old                       105.229004   
                         Seltjarnarnesbær        new                       116.186667   
                                                 old                       105.643750   

                                                                           rooms  \
realestate_category_name city_name               construction_category             
Fjölbýli                 Garðabær                new                    3.250000   
                                                 old                    3.377049   
                         Hafnarfjarðarkaupstaður new                    3.118987   
                                                 old                    3.290640   
                         Kópavogsbær             new                    3.312500   
                                                 old                    3.382883   
                         Mosfellsbær             new                    3.000000   
                                                 old                    3.421053   
                         Reykjavíkurborg         new                    3.030137   
                                                 old                    3.322425   
                         Seltjarnarnesbær        new                    3.333333   
                                                 old                    3.062500   

                                                                        construction_year  \
realestate_category_name city_name               construction_category                      
Fjölbýli                 Garðabær                new                          2024.731481   
                                                 old                          2011.081967   
                         Hafnarfjarðarkaupstaður new                          2023.865823   
                                                 old                          2001.610837   
                         Kópavogsbær             new                          2024.660714   
                                                 old                          1999.207207   
                         Mosfellsbær             new                          2024.000000   
                                                 old                          2014.736842   
                         Reykjavíkurborg         new                          2024.369863   
                                                 old                          1979.998936   
                         Seltjarnarnesbær        new                          2024.466667   
                                                 old                          1988.062500   

                                                                               price  \
realestate_category_name city_name               construction_category                 
Fjölbýli                 Garðabær                new                    1.262280e+08   
                                                 old                    1.047736e

In [ ]:
df_sales\
.loc[lambda r: r.square_meters < 300]\
.loc[lambda r: r.realestate_category_name == 'Fjölbýli']\
.assign(sqm_price = lambda r: r.purchase_price / r.square_meters)\
.groupby(['realestate_category_name', 'city_name', 'construction_category'])\
.agg(
    square_meters = ('square_meters', 'mean'),
    rooms = ('rooms', 'mean'),
    construction_year = ('construction_year', 'mean'),
    purchase_price = ('purchase_price', 'mean'),
    total = ('realestate_id', 'count'),
    sqm_price = ('sqm_price', 'mean'),
    listing_days = ('listing_days', 'mean')
)

square_meters  \
realestate_category_name city_name               construction_category                  
Fjölbýli                 Garðabær                new                        97.898333   
                                                 old                       110.569042   
                         Hafnarfjarðarkaupstaður new                        99.128571   
                                                 old                        97.598498   
                         Kópavogsbær             new                        95.648276   
                                                 old                       103.020554   
                         Mosfellsbær             new                        90.305000   
                                                 old                        98.527119   
                         Reykjavíkurborg         new                        81.536478   
                                                 old                        96.818175   
                         Seltjarnarnesbær        new                       110.975000   
                                                 old                       114.036585   

                                                                           rooms  \
realestate_category_name city_name               construction_category             
Fjölbýli                 Garðabær                new                    2.933333   
                                                 old                    3.316953   
                         Hafnarfjarðarkaupstaður new                    3.222222   
                                                 old                    3.174174   
                         Kópavogsbær             new                    2.948276   
                                                 old                    3.323615   
                         Mosfellsbær             new                    2.650000   
                                                 old                    3.203390   
                         Reykjavíkurborg         new                    2.754717   
                                                 old                    3.262914   
                         Seltjarnarnesbær        new                    3.750000   
                                                 old                    3.487805   

                                                                        construction_year  \
realestate_category_name city_name               construction_category                      
Fjölbýli                 Garðabær                new                          2021.916667   
                                                 old                          2014.882064   
                         Hafnarfjarðarkaupstaður new                          2022.047619   
                                                 old                          1998.277778   
                         Kópavogsbær             new                          2021.465517   
                                                 old                          1995.696793   
                         Mosfellsbær             new                          2020.550000   
                                                 old                          2009.214689   
                         Reykjavíkurborg         new                          2021.716981   
                                                 old                          1975.361358   
                         Seltjarnarnesbær        new                          2024.500000   
                                                 old                          1988.036585   

                                                                        purchase_price  \
realestate_category_name city_name               construction_category                   
Fjölbýli                 Garðabær                new                      8.458583e+07   
                                                 old                      9

In [ ]:
df_listings\
.loc[lambda r: r.square_meters < 300]\
.loc[lambda r: r.realestate_category_name == 'Fjölbýli']\
.assign(sqm_price = lambda r: r.price / r.square_meters)\
.groupby(['realestate_category_name', 'city_name', 'construction_category'])\
.agg(
    square_meters = ('square_meters', 'mean'),
    rooms = ('rooms', 'mean'),
    construction_year = ('construction_year', 'mean'),
    price = ('price', 'mean'),
    total = ('realestate_id', 'count'),
    sqm_price = ('sqm_price', 'mean'),
    days_listed = ('days_listed', 'mean')
)

square_meters  \
realestate_category_name city_name               construction_category                  
Fjölbýli                 Garðabær                new                       124.804630   
                                                 old                       120.919672   
                         Hafnarfjarðarkaupstaður new                        99.571899   
                                                 old                        98.196552   
                         Kópavogsbær             new                       105.545536   
                                                 old                       114.467568   
                         Mosfellsbær             new                        93.566667   
                                                 old                       102.468421   
                         Reykjavíkurborg         new                       105.063288   
                                                 old                       105.154044   
                         Seltjarnarnesbær        new                       116.186667   
                                                 old                       105.643750   

                                                                           rooms  \
realestate_category_name city_name               construction_category             
Fjölbýli                 Garðabær                new                    3.250000   
                                                 old                    3.377049   
                         Hafnarfjarðarkaupstaður new                    3.118987   
                                                 old                    3.290640   
                         Kópavogsbær             new                    3.312500   
                                                 old                    3.382883   
                         Mosfellsbær             new                    3.000000   
                                                 old                    3.421053   
                         Reykjavíkurborg         new                    3.030137   
                                                 old                    3.320192   
                         Seltjarnarnesbær        new                    3.333333   
                                                 old                    3.062500   

                                                                        construction_year  \
realestate_category_name city_name               construction_category                      
Fjölbýli                 Garðabær                new                          2024.731481   
                                                 old                          2011.081967   
                         Hafnarfjarðarkaupstaður new                          2023.865823   
                                                 old                          2001.610837   
                         Kópavogsbær             new                          2024.660714   
                                                 old                          1999.207207   
                         Mosfellsbær             new                          2024.000000   
                                                 old                          2014.736842   
                         Reykjavíkurborg         new                          2024.369863   
                                                 old                          1979.998937   
                         Seltjarnarnesbær        new                          2024.466667   
                                                 old                          1988.062500   

                                                                               price  \
realestate_category_name city_name               construction_category                 
Fjölbýli                 Garðabær                new                    1.262280e+08   
                                                 old                    1.047736e